In [1]:
from io import BytesIO
import xarray as xr
import matplotlib

matplotlib.use("Agg")
import matplotlib.pyplot as plt
import imageio.v3 as iio
from dask.distributed import Client


def process(ds_sel, time):
    fig, ax = plt.subplots()
    ds_sel["air"].plot(ax=ax, vmin=220, vmax=340)
    with BytesIO() as buf:
        fig.savefig(buf, format="png")
        plt.close(fig)
        image = iio.imread(buf.getvalue())
    return image

/g/data/xp65/public/apps/med_conda/envs/analysis3-25.10/lib/python3.11/site-packages/distributed/diagnostics/nvml.py:14: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml


In [ ]:
client = Client()
display(client)

In [4]:
%%time
ds = xr.tutorial.open_dataset("air_temperature").isel(time=slice(0, 1000))
futures = [
    client.submit(process, ds.sel(time=time), time) for time in ds["time"].values
]

with iio.imopen("/home/566/nd0349/access-om3-analysis/figures/test.gif", "w", extension=".gif") as f:
    for future in futures:
        image = future.result()
        f.write(image)

CPU times: user 25.9 s, sys: 7.28 s, total: 33.1 s
Wall time: 31.9 s
